# Deep denoising auto-encoder and MLP based multi-output regression on TCGA multi-omics data
## DNA Methylation and CNA to RNA-Seq

# Setting environment

Seeding the random number generators

In [0]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # have reproducible behavior for certain hash-based operations.
import os
os.environ['PYTHONHASHSEED'] = '0'
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
import numpy as np
np.random.seed(42)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
import random as rn
rn.seed(12345)

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
import tensorflow as tf
tf.set_random_seed(1234)

# Force TensorFlow to use single thread.
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Importing libraries

In [0]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

In [0]:
def rSquared(true,predicted):
    cols = predicted.shape[1]
    rsq = np.zeros(shape=(cols), dtype = np.float32)
    for j in range(cols):
        rsq[j] = r2_score(true[:,j], predicted[:,j])
    return rsq

# Loading data

Importing data from pre-processed csv files (Change paths accordingly)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#ls "/content/drive/My Drive"

In [0]:
preprocessed_DNAMeth = pd.read_csv('/content/drive/My Drive/TCGA Data/Preprocessed_Data/LIHC_preprocessed_DNAMeth.csv')
preprocessed_RNASeq = pd.read_csv('/content/drive/My Drive/TCGA Data/Preprocessed_Data/LIHC_preprocessed_RNASeq.csv')
preprocessed_CNA = pd.read_csv('/content/drive/My Drive/TCGA Data/Preprocessed_Data/LIHC_preprocessed_CNA.csv')
labels = pd.read_csv('/content/drive/My Drive/TCGA Data/Preprocessed_Data/LIHC_labels.csv')

In [0]:
x1 = preprocessed_DNAMeth
x2 = preprocessed_CNA    
y = preprocessed_RNASeq

Concatenating Methylation and CNV features

In [0]:
x1 = pd.DataFrame(x1)
x2 = pd.DataFrame(x2)
df = [x1, x2]
z = pd.concat(df,axis=1)

Splitting the data into training and testing datasets

In [0]:
x_train, x_test, y_train, y_test, labels_train, labels_test = train_test_split(z, y, labels, test_size=0.2)

Scaling the data within [0-1] range

In [0]:
scalar = MinMaxScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

Adding gaussian noise

In [0]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(0.0, 1.0, x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(0.0, 1.0, x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Dimension Reduction/Feature Extraction using DDAE

Setting the no. of input and output neurons

In [0]:
num_in_neurons = z.shape[1]
num_out_neurons = y.shape[1]

In [0]:
# Auto-encoder to extract features from DNA Methylation and CNV data

with tf.device('/gpu:0'):
    # this is the size of our encoded representations
    encoding_dim1 = 500
    encoding_dim2 = 200
    
    lambda_act = 0.0001
    lambda_weight = 0.001
    # this is our input placeholder
    input_data = Input(shape=(num_in_neurons,))
    # first encoded representation of the input
    encoded = Dense(encoding_dim1, activation='relu', activity_regularizer=regularizers.l1(lambda_act), kernel_regularizer=regularizers.l2(lambda_weight), name='encoder1')(input_data)
    # second encoded representation of the input
    encoded = Dense(encoding_dim2, activation='relu', activity_regularizer=regularizers.l1(lambda_act), kernel_regularizer=regularizers.l2(lambda_weight), name='encoder2')(encoded)
    # first lossy reconstruction of the input
    decoded = Dense(encoding_dim1, activation='relu', name='decoder1')(encoded)
    # the final lossy reconstruction of the input
    decoded = Dense(num_in_neurons, activation='sigmoid', name='decoder2')(decoded)
    
    # this model maps an input to its reconstruction
    autoencoder = Model(inputs=input_data, outputs=decoded)
    
    myencoder = Model(inputs=input_data, outputs=encoded)
    autoencoder.compile(optimizer='sgd', loss='mse')
    # training
    print('training the autoencoder')
    autoencoder.fit(x_train_noisy, x_train,
                    epochs=25,
                    batch_size=8,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test))
    autoencoder.trainable = False   #freeze autoencoder weights

# Regression using MLP

In [0]:
# MLP Multi-output Regression code goes here...

num_hidden = encoding_dim2
with tf.device('/gpu:0'):    
    x = autoencoder.get_layer('encoder2').output
    x = Dropout(0.2)(x)             # adding 20% dropout
    h = Dense(int(num_hidden * 3), activation='relu', name='hidden1')(x)
    h = Dropout(0.5)(h)             # adding 50% dropout
    h = Dense(int(num_hidden * 5), activation='relu', name='hidden2')(h)
    h = Dropout(0.5)(h)             # adding 50% dropout
    y = Dense(num_out_neurons, activation='linear', name='prediction')(h)
    mlpRegressor = Model(inputs=autoencoder.inputs, outputs=y)

    # Compile model
    mlpRegressor.compile(loss='mse', optimizer='adam', metrics=['accuracy'])    # or loss='mae'
    
    # Fit the model
    print('training the MLP multi-output regressor')
    mlpRegressor.fit(x_train, y_train, epochs=50, batch_size=8)
    
    y_pred = mlpRegressor.predict(x_test)
    
    actual_mean = pd.DataFrame(y_test.mean(axis=0))
    pred_mean = pd.DataFrame(y_pred.mean(axis=0))

# Results

In [0]:
print('MSE: (Actual Vs. Predicted)', mean_squared_error(y_test, y_pred))
print('r^2 value: (Mean of actual Vs. Mean of Predicted)', r2_score(actual_mean, pred_mean))

Plotting predicted Vs. Actual

In [0]:
act=actual_mean.values.flatten()
pred=pred_mean.values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Average of actual and predicted gene expression values across all samples')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('DDAE-MLP')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

Plotting correlation scatter plot for mean of actual Vs. mean of predicted gene expressions

In [0]:
plt.figure(figsize=(20,10))
plt.scatter(actual_mean, pred_mean)
plt.title('Correlation between mean of actual and mean predicted gene expression across all samples')
plt.xlabel('Average of true gene expressions across samples')
plt.ylabel('Average of predicted gene expressions across samples')
plt.grid(True)
plt.show()


# Classification of Tumor and Normal samples using MLP

Importing libraries

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

MLP-classifier

In [0]:
num_hidden = encoding_dim2
with tf.device('/gpu:0'):    
    x = autoencoder.get_layer('encoder2').output
    x = Dropout(0.2)(x)     # adding 20% dropout
    h = Dense(int(num_hidden * 3), activation='relu', name='hidden1')(x)
    h = Dropout(0.5)(h)     # adding 50% dropout
    h = Dense(int(num_hidden * 5), activation='relu', name='hidden2')(h)
    h = Dropout(0.5)(h)     # adding 50% dropout
    y = Dense(1, activation='sigmoid', name='predictions')(h)

    classifier = Model(inputs=autoencoder.inputs, outputs=y)
    # Compile model
    classifier.compile(loss='binary_crossentropy', optimizer='adam',
                   metrics=['accuracy'])
    # Fit the model
    classifier.fit(x_train, labels_train, epochs=25, batch_size=8)

    print('Now making predictions')
    predictions = classifier.predict(x_test)
    rounded_predictions = [round(x[0]) for x in predictions]

Evaluating the model

In [0]:
_, train_acc = classifier.evaluate(x_train, labels_train, verbose=0)
_, test_acc = classifier.evaluate(x_test, labels_test, verbose=0)
print('\nTraining accuracy: %.3f, Testing accuracy: %.3f' % (train_acc, test_acc))
print("Recall score = ",recall_score(labels_test, rounded_predictions))
cm = confusion_matrix(labels_test, rounded_predictions)
print("Confusion matrix:")
print(cm)
report = classification_report(labels_test, rounded_predictions)
print("classification_report")
print(report)

# Comparing regression results with other standard methods

Evaluating the mean of actual y values

In [0]:
actual_mean = pd.DataFrame(y_test.mean(axis=0))

# 1. Linear Regression

Importing libraries

In [0]:
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

Multi-output regression using Linear Regression (OLS) (sk-learn)

In [0]:
with tf.device('/gpu:0'):
    linear_Regr = LinearRegression(normalize=True)
    linear_Regr.fit(x_train, y_train)
    y_pred = linear_Regr.predict(x_test)
    pred_mean = pd.DataFrame(y_pred.mean(axis=0))
    y_mse = mean_squared_error(y_test, y_pred)
    y_r2score = r2_score(actual_mean, pred_mean)

In [0]:
print("Mean Squared Error (y_test Vs. y_pred): ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('Linear Regression')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# 2. Lasso

Importing libraries

In [0]:
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

Multi-output regression using Lasso (sk-learn)

In [0]:
y_mse=[]
y_r2score=[]
with tf.device('/gpu:0'):
    #for alp in [0.1, 0.2, 0.3, 0.4, 0.5]
    for alp in [0.01,0.1,0.5,1,5]:
        print('Working with alpha=',alp)
        Lasso_Regr = Lasso(alpha=alp, normalize=True, random_state=42)
        Lasso_Regr.fit(x_train, y_train)
        y_pred = Lasso_Regr.predict(x_test)
        pred_mean = pd.DataFrame(y_pred.mean(axis=0))
        y_mse.append(mean_squared_error(y_test, y_pred))
        y_r2score.append(r2_score(actual_mean, pred_mean))

In [0]:
print("Mean Squared Error (y_test Vs. y_pred): ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('Lasso with alpha = 1.5')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# 3. Ridge

Importing libraries

In [0]:
from sklearn.linear_model import Ridge

Multi-output regression using Ridge (sk-learn)

In [0]:
y_mse=[]
y_r2score=[]
with tf.device('/gpu:0'):
    for alp in [0.01,0.1,0.5,1,1.5]:
        print('Working with alpha = ',alp)
        Ridge_Regr = Ridge(alpha=alp, normalize=True)
        Ridge_Regr.fit(x_train, y_train)
        y_pred = Ridge_Regr.predict(x_test)
        pred_mean = pd.DataFrame(y_pred.mean(axis=0))
        y_mse.append(mean_squared_error(y_test, y_pred))
        y_r2score.append(r2_score(actual_mean, pred_mean))

In [0]:
print("Mean Squared Error (y_test Vs. y_pred): ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('Ridge Regression with alpha=1.5')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# 4. PCA - Random Forest (PCA-RF)

Importing libraries

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

In [0]:
n=200
pca = PCA(n_components=n)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

Multi-output regression using Random Forest (sk-learn)

In [0]:
y_mse=[]
y_r2score=[]
with tf.device('/gpu:0'):
    for est in [10,50,100,150,200]:
        print('estimators = ',est)
        rf_Regr = RandomForestRegressor(n_estimators=est, n_jobs=-1)
        rf_Regr.fit(x_train, y_train)
        y_pred = rf_Regr.predict(x_test)
        pred_mean = pd.DataFrame(y_pred.mean(axis=0))
        y_mse.append(mean_squared_error(y_test, y_pred))
        y_r2score.append(r2_score(actual_mean, pred_mean))

In [0]:
print("Mean Squared Error: ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('PCA-RF with 250 features and 100 estimators')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# 5. k-Nearest Neighbor (kNN)

Importing libraries

In [0]:
from sklearn.neighbors import KNeighborsRegressor

Multi-output regression using kNN (sk-learn)

In [0]:
y_mse=[]
y_r2score=[]
with tf.device('/gpu:0'):
    for k in [5,10,15,20,25]:
        print('k=',k)
        knn_Regr = KNeighborsRegressor(n_neighbors=k, n_jobs=-1)
        knn_Regr.fit(x_train, y_train)
        y_pred = knn_Regr.predict(x_test)
        pred_mean = pd.DataFrame(y_pred.mean(axis=0))
        y_mse.append(mean_squared_error(y_test, y_pred))
        y_r2score.append(r2_score(actual_mean, pred_mean))

In [0]:
print("Mean Squared Error: ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('k-NN with k=10')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# 6. PCA - Support Vector Regression (PCA-SVR)

In [0]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA

In [0]:
n=200
pca = PCA(n_components=n)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

In [0]:
with tf.device('/gpu:0'):
    y_mse=[]
    y_r2score = []
    for k in ['linear','poly','rbf','sigmoid']:
        print('kernel = ',k)
        mo_svr = MultiOutputRegressor(SVR(kernel=k,gamma='auto'))
        mo_svr.fit(x_train, y_train)
        y_pred = mo_svr.predict(x_test)
        pred_mean = pd.DataFrame(y_pred.mean(axis=0))
        y_mse.append(mean_squared_error(y_test, y_pred))
        y_r2score.append(r2_score(actual_mean, pred_mean))

In [0]:
print("Mean Squared Error: ", y_mse)
print("r2 Score (y_test_mean Vs. y_pred_mean): ", y_r2score)

Plotting first 100 features

In [0]:
act=actual_mean[0:100].values.flatten()
pred=pred_mean[0:100].values.flatten()

s1 = pd.Series(act)
s2 = pd.Series(pred)

plt.figure(figsize=(10,5))
ax = plt.subplot(111)
plt.title('PCA-RF with 250 features and linear kernel')
plt.xlabel('No. of features (genes)')
plt.ylabel('Average of gene expression values across samples')
ax.plot(s1, 'b--', label='Actual')
ax.plot(s2, 'r--', label='Predicted')
ax.legend()
plt.grid(True)
plt.show()

# Comparison regression results from AE-MLP with PCA-MLP

Importing libraries

In [0]:
from sklearn.decomposition import PCA

PCA-MLP

In [0]:
n=100
with tf.device('/gpu:0'):
    #pca
    pca = PCA(n_components=n)
    pca.fit(x_train)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    
    # MLP Multi-output Regression code goes here...
    num = n
    input_data = Input(shape=(num,))
    x = Dropout(0.2)(input_data)             # adding 20% dropout
    h = Dense(int(num * 3), activation='relu', name='hidden1')(x)
    h = Dropout(0.5)(h)                        # adding 50% dropout
    h = Dense(int(num * 5), activation='relu', name='hidden2')(h)
    h = Dropout(0.5)(h)                        # adding 50% dropout
    y = Dense(num_out_neurons, activation='linear', name='prediction')(h)
    mlpRegressor = Model(inputs=input_data, outputs=y)
    
    # Compile model
    mlpRegressor.compile(loss='mse', optimizer='adam', metrics=['accuracy'])    # or loss='mae'
    # Fit the model
    print('training the MLP multi-output regressor')
    mlpRegressor.fit(x_train, y_train, epochs=50, batch_size=8)
    y_pred = mlpRegressor.predict(x_test)
    actual_mean = pd.DataFrame(y_test.mean(axis=0))
    pred_mean = pd.DataFrame(y_pred.mean(axis=0))

Printing results

In [0]:
print('MSE: (Actual Vs. Predicted)', mean_squared_error(y_test, y_pred))
print('r^2 value: (Mean of actual Vs. Mean of Predicted)', r2_score(actual_mean, pred_mean))